# Regresión Lineal Múltiple: Correlación, VIF y Regularización

En este ejercicio trabajaremos con el dataset de Boston Housing para:

1. **Estudiar la correlación entre variables**
2. **Calcular el VIF (Variance Inflation Factor)** de cada variable
3. **Analizar el impacto de eliminar variables correlacionadas** en las métricas del modelo
4. **Aplicar regularización L1 (Lasso) y L2 (Ridge)** y analizar sus efectos
5. **Interpretar los resultados** de las diferentes técnicas

## Importar las librerías necesarias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# Configuración de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

## Cargar y explorar el dataset

In [ ]:
# Cargar el dataset de Boston Housing
dataset = pd.read_csv('../data/BostonHousing.csv')

# Mostrar información básica del dataset
print("Dimensiones del dataset:", dataset.shape)
print("\nPrimeras 5 filas:")
dataset.head()

In [ ]:
# Información sobre las variables del dataset
print("Información del dataset:")
dataset.info()

print("\nEstadísticas descriptivas:")
dataset.describe()

### Descripción de las variables:

- **CRIM**: Tasa de criminalidad per cápita por ciudad
- **ZN**: Proporción de terrenos residenciales zonificados para lotes de más de 25,000 pies cuadrados
- **INDUS**: Proporción de acres de negocios no minoristas por ciudad
- **CHAS**: Variable binaria del río Charles (1 si limita con el río, 0 en caso contrario)
- **NOX**: Concentración de óxidos nítricos (partes por 10 millones)
- **RM**: Número promedio de habitaciones por vivienda
- **AGE**: Proporción de unidades ocupadas por el propietario construidas antes de 1940
- **DIS**: Distancias ponderadas a cinco centros de empleo de Boston
- **RAD**: Índice de accesibilidad a las autopistas radiales
- **TAX**: Tasa de impuesto a la propiedad de valor completo por $10,000
- **PTRATIO**: Relación alumno-maestro por ciudad
- **B**: 1000(Bk - 0.63)^2 donde Bk es la proporción de personas de raza negra por ciudad
- **LSTAT**: % de estatus socioeconómico más bajo de la población
- **MEDV**: Valor mediano de las viviendas ocupadas por el propietario en $1000s (VARIABLE OBJETIVO)

## 1. Análisis de Correlación entre Variables

In [ ]:
# Calcular la matriz de correlación
correlation_matrix = dataset.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdBu_r',
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={"shrink": .8})
plt.title('Matriz de Correlación - Boston Housing Dataset', fontsize=16, pad=20)
plt.show()

In [ ]:
# Identificar correlaciones altas (> 0.7 o < -0.7) excluyendo la diagonal
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append({
                'Variable 1': correlation_matrix.columns[i],
                'Variable 2': correlation_matrix.columns[j],
                'Correlación': correlation_matrix.iloc[i, j]
            })

high_corr_df = pd.DataFrame(high_corr_pairs)
if not high_corr_df.empty:
    high_corr_df = high_corr_df.sort_values('Correlación', key=abs, ascending=False)
    print("Pares de variables con correlación alta (|r| > 0.7):")
    print(high_corr_df)
else:
    print("No se encontraron correlaciones altas (|r| > 0.7)")

In [ ]:
# Correlación de cada variable con la variable objetivo (MEDV)
target_correlation = correlation_matrix['MEDV'].drop('MEDV').sort_values(key=abs, ascending=False)

plt.figure(figsize=(10, 8))
colors = ['red' if x < 0 else 'blue' for x in target_correlation.values]
plt.barh(range(len(target_correlation)), target_correlation.values, color=colors, alpha=0.7)
plt.yticks(range(len(target_correlation)), target_correlation.index)
plt.xlabel('Correlación con MEDV (Variable Objetivo)')
plt.title('Correlación de cada Variable con el Precio de la Vivienda (MEDV)')
plt.grid(axis='x', alpha=0.3)
for i, v in enumerate(target_correlation.values):
    plt.text(v + 0.01 if v > 0 else v - 0.01, i, f'{v:.3f}', 
             va='center', ha='left' if v > 0 else 'right')
plt.tight_layout()
plt.show()

print("\nCorrelaciones con la variable objetivo (MEDV):")
print(target_correlation)

## 2. Cálculo del VIF (Variance Inflation Factor)

In [ ]:
# Preparar los datos para el cálculo del VIF
X = dataset.drop('MEDV', axis=1)
y = dataset['MEDV']

# Agregar constante para el cálculo del VIF
X_with_const = sm.add_constant(X)

# Calcular VIF para cada variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i+1) 
                   for i in range(len(X.columns))]

# Ordenar por VIF descendente
vif_data = vif_data.sort_values('VIF', ascending=False)

print("Factor de Inflación de la Varianza (VIF) para cada variable:")
print(vif_data)
print("\nInterpretación del VIF:")
print("- VIF = 1: No hay multicolinealidad")
print("- VIF > 5: Multicolinealidad moderada")
print("- VIF > 10: Multicolinealidad severa")

In [ ]:
# Visualizar los valores de VIF
plt.figure(figsize=(12, 8))
colors = ['red' if x > 10 else 'orange' if x > 5 else 'green' for x in vif_data['VIF']]
bars = plt.bar(vif_data['Variable'], vif_data['VIF'], color=colors, alpha=0.7)
plt.axhline(y=5, color='orange', linestyle='--', alpha=0.7, label='VIF = 5 (Umbral moderado)')
plt.axhline(y=10, color='red', linestyle='--', alpha=0.7, label='VIF = 10 (Umbral severo)')
plt.xticks(rotation=45, ha='right')
plt.ylabel('VIF (Factor de Inflación de la Varianza)')
plt.title('VIF por Variable - Detección de Multicolinealidad')
plt.legend()
plt.grid(axis='y', alpha=0.3)

# Añadir valores sobre las barras
for bar, vif_val in zip(bars, vif_data['VIF']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{vif_val:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 3. Modelo de Referencia (Sin Eliminación de Variables)

In [ ]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal múltiple
model_base = LinearRegression()
model_base.fit(X_train, y_train)

# Realizar predicciones
y_pred_base = model_base.predict(X_test)

# Calcular métricas del modelo base
mse_base = mean_squared_error(y_test, y_pred_base)
rmse_base = np.sqrt(mse_base)
r2_base = r2_score(y_test, y_pred_base)
mae_base = mean_absolute_error(y_test, y_pred_base)

print("=== MODELO BASE (Todas las variables) ===")
print(f"MSE: {mse_base:.3f}")
print(f"RMSE: {rmse_base:.3f}")
print(f"R²: {r2_base:.3f}")
print(f"MAE: {mae_base:.3f}")

# Almacenar métricas para comparación posterior
results_comparison = pd.DataFrame({
    'Modelo': ['Base (Todas las variables)'],
    'MSE': [mse_base],
    'RMSE': [rmse_base],
    'R²': [r2_base],
    'MAE': [mae_base],
    'Num_Variables': [X.shape[1]]
})

## 4. Eliminación de Variables con VIF Alto

In [ ]:
# TODO: Identificar variables con VIF > 10 para eliminar
# Sugerencia: Usa vif_data para identificar las variables problemáticas

high_vif_vars = # COMPLETAR: variables con VIF > 10

print(f"Variables con VIF > 10 a eliminar: {high_vif_vars}")

In [ ]:
# TODO: Crear dataset sin las variables de VIF alto
# Sugerencia: Usa drop() para eliminar las variables identificadas

X_reduced = # COMPLETAR: eliminar variables con VIF alto

print(f"Número de variables después de eliminación: {X_reduced.shape[1]}")
print(f"Variables eliminadas: {set(X.columns) - set(X_reduced.columns)}")

In [ ]:
# TODO: Recalcular VIF para el dataset reducido
# Sugerencia: Repite el proceso de cálculo de VIF con X_reduced

# COMPLETAR: calcular VIF para X_reduced

print("VIF después de eliminar variables problemáticas:")
# COMPLETAR: mostrar los nuevos valores de VIF

In [ ]:
# TODO: Entrenar modelo con variables reducidas
# Sugerencia: Divide X_reduced en train/test y entrena un nuevo modelo

X_reduced_train, X_reduced_test, y_train_reduced, y_test_reduced = # COMPLETAR

model_reduced = # COMPLETAR: crear y entrenar modelo

# TODO: Calcular métricas del modelo reducido
y_pred_reduced = # COMPLETAR

# COMPLETAR: calcular MSE, RMSE, R², MAE

print("=== MODELO REDUCIDO (Sin variables VIF alto) ===")
# COMPLETAR: mostrar métricas

In [ ]:
# Agregar resultados del modelo reducido a la comparación
# TODO: Completar este código con las métricas calculadas

new_row = pd.DataFrame({
    'Modelo': ['Reducido (Sin VIF alto)'],
    'MSE': [mse_reduced],  # COMPLETAR
    'RMSE': [rmse_reduced],  # COMPLETAR
    'R²': [r2_reduced],  # COMPLETAR
    'MAE': [mae_reduced],  # COMPLETAR
    'Num_Variables': [X_reduced.shape[1]]
})

results_comparison = pd.concat([results_comparison, new_row], ignore_index=True)

## 5. Regularización L1 (Lasso) y L2 (Ridge)

In [ ]:
# Estandarizar los datos para regularización
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Datos estandarizados para regularización")
print(f"Media de X_train_scaled: {np.mean(X_train_scaled, axis=0).round(3)}")
print(f"Std de X_train_scaled: {np.std(X_train_scaled, axis=0).round(3)}")

### 5.1 Regresión Ridge (L2)

In [ ]:
# TODO: Probar diferentes valores de alpha para Ridge
# Sugerencia: Usa un rango de valores como [0.1, 1, 10, 100, 1000]

alpha_values = # COMPLETAR: definir valores de alpha
ridge_results = []

for alpha in alpha_values:
    # TODO: Crear y entrenar modelo Ridge
    ridge_model = # COMPLETAR
    
    # TODO: Hacer predicciones y calcular métricas
    y_pred_ridge = # COMPLETAR
    
    mse_ridge = # COMPLETAR
    r2_ridge = # COMPLETAR
    
    ridge_results.append({
        'Alpha': alpha,
        'MSE': mse_ridge,
        'R²': r2_ridge
    })

ridge_results_df = pd.DataFrame(ridge_results)
print("Resultados de Ridge Regression:")
print(ridge_results_df)

### 5.2 Regresión Lasso (L1)

In [ ]:
# TODO: Probar diferentes valores de alpha para Lasso
# Sugerencia: Usa valores más pequeños como [0.01, 0.1, 1, 10, 100]

lasso_results = []

for alpha in alpha_values:
    # TODO: Crear y entrenar modelo Lasso
    lasso_model = # COMPLETAR
    
    # TODO: Hacer predicciones y calcular métricas
    y_pred_lasso = # COMPLETAR
    
    mse_lasso = # COMPLETAR
    r2_lasso = # COMPLETAR
    
    # Contar variables seleccionadas (coeficientes no cero)
    selected_features = # COMPLETAR: suma de coeficientes != 0
    
    lasso_results.append({
        'Alpha': alpha,
        'MSE': mse_lasso,
        'R²': r2_lasso,
        'Variables_Seleccionadas': selected_features
    })

lasso_results_df = pd.DataFrame(lasso_results)
print("Resultados de Lasso Regression:")
print(lasso_results_df)

### 5.3 Comparación Visual de Regularización

In [ ]:
# TODO: Crear gráficos comparativos
# Sugerencia: Grafica MSE vs Alpha para Ridge y Lasso

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: MSE vs Alpha
# COMPLETAR: plotear ridge_results_df y lasso_results_df

# Gráfico 2: R² vs Alpha  
# COMPLETAR: plotear R² para ambos modelos

plt.tight_layout()
plt.show()

### 5.4 Análisis de Coeficientes

In [ ]:
# TODO: Seleccionar mejores alphas y comparar coeficientes
# Sugerencia: Elige el alpha con mejor R² para cada método

best_ridge_alpha = # COMPLETAR: mejor alpha para Ridge
best_lasso_alpha = # COMPLETAR: mejor alpha para Lasso

# Entrenar modelos finales
final_ridge = Ridge(alpha=best_ridge_alpha)
final_lasso = Lasso(alpha=best_lasso_alpha)

final_ridge.fit(X_train_scaled, y_train)
final_lasso.fit(X_train_scaled, y_train)

# Comparar coeficientes
coefficients_comparison = pd.DataFrame({
    'Variable': X.columns,
    'Linear_Regression': model_base.coef_,
    'Ridge': final_ridge.coef_,
    'Lasso': final_lasso.coef_
})

print("Comparación de coeficientes:")
print(coefficients_comparison.round(3))

In [ ]:
# TODO: Visualizar coeficientes
# Sugerencia: Crea un gráfico de barras comparando los coeficientes

plt.figure(figsize=(14, 8))
x_pos = np.arange(len(X.columns))
width = 0.25

# COMPLETAR: crear gráfico de barras con los tres tipos de coeficientes

plt.title('Comparación de Coeficientes: Linear vs Ridge vs Lasso')
plt.xlabel('Variables')
plt.ylabel('Valor del Coeficiente')
plt.xticks(x_pos, X.columns, rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Comparación Final de Modelos

In [ ]:
# TODO: Agregar resultados de regularización a la comparación
# Sugerencia: Calcula métricas para los modelos Ridge y Lasso finales

# Predicciones de modelos regularizados
y_pred_ridge_final = # COMPLETAR
y_pred_lasso_final = # COMPLETAR

# Métricas de Ridge
mse_ridge_final = # COMPLETAR
r2_ridge_final = # COMPLETAR
rmse_ridge_final = # COMPLETAR
mae_ridge_final = # COMPLETAR

# Métricas de Lasso  
mse_lasso_final = # COMPLETAR
r2_lasso_final = # COMPLETAR
rmse_lasso_final = # COMPLETAR
mae_lasso_final = # COMPLETAR

# Agregar a comparación
ridge_row = pd.DataFrame({
    'Modelo': [f'Ridge (α={best_ridge_alpha})'],
    'MSE': [mse_ridge_final],
    'RMSE': [rmse_ridge_final],
    'R²': [r2_ridge_final],
    'MAE': [mae_ridge_final],
    'Num_Variables': [X.shape[1]]  # Ridge no elimina variables
})

lasso_row = pd.DataFrame({
    'Modelo': [f'Lasso (α={best_lasso_alpha})'],
    'MSE': [mse_lasso_final],
    'RMSE': [rmse_lasso_final],
    'R²': [r2_lasso_final],
    'MAE': [mae_lasso_final],
    'Num_Variables': [np.sum(final_lasso.coef_ != 0)]
})

results_comparison = pd.concat([results_comparison, ridge_row, lasso_row], ignore_index=True)

In [ ]:
# Mostrar comparación final
print("=== COMPARACIÓN FINAL DE MODELOS ===")
print(results_comparison.round(3))

# Identificar el mejor modelo
best_model_idx = results_comparison['R²'].idxmax()
best_model = results_comparison.loc[best_model_idx, 'Modelo']
best_r2 = results_comparison.loc[best_model_idx, 'R²']

print(f"\nMejor modelo basado en R²: {best_model} (R² = {best_r2:.3f})")

In [ ]:
# TODO: Crear visualización de comparación de modelos
# Sugerencia: Gráfico de barras con las métricas principales

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

models = results_comparison['Modelo']

# COMPLETAR: crear 4 subgráficos para MSE, RMSE, R², y MAE

plt.suptitle('Comparación de Rendimiento de Modelos', fontsize=16)
plt.tight_layout()
plt.show()

## 7. Interpretación y Conclusiones

### TODO: Completa el análisis respondiendo a estas preguntas:

#### 7.1 Análisis de Correlación:
- ¿Qué variables muestran mayor correlación con el precio de la vivienda (MEDV)?
- ¿Qué pares de variables están más correlacionadas entre sí?
- ¿Cómo podrías interpretar estas correlaciones desde el punto de vista del negocio inmobiliario?

#### 7.2 Análisis del VIF:
- ¿Qué variables presentan problemas de multicolinealidad?
- ¿Cómo afectó la eliminación de variables con VIF alto al rendimiento del modelo?
- ¿Vale la pena sacrificar algunas variables para reducir la multicolinealidad?

#### 7.3 Efectos de la Regularización:
- ¿Cómo se comparan Ridge y Lasso en términos de rendimiento?
- ¿Qué variables selecciona Lasso como más importantes?
- ¿Qué diferencias observas en los coeficientes entre los métodos?
- ¿Cuándo recomendarías usar cada tipo de regularización?

#### 7.4 Recomendación Final:
- ¿Qué modelo recomendarías para predecir precios de viviendas en Boston?
- ¿Qué factores considerarías además del rendimiento estadístico?
- ¿Cómo explicarías los resultados a un stakeholder no técnico?

### Respuestas:

**7.1 Análisis de Correlación:**
TODO: Completa con tus observaciones

**7.2 Análisis del VIF:**
TODO: Completa con tus observaciones

**7.3 Efectos de la Regularización:**
TODO: Completa con tus observaciones

**7.4 Recomendación Final:**
TODO: Completa con tus recomendaciones